# Calculations for the Frontiers 2021 paper

(computed post-acceptance)

In [1]:
import math
import pandas as pd
import re 

## 1. Recompute Table 1 percentages

In [2]:
table_dict = {         "males": { 1: 209, 2: 133 },
                     "females": { 1: 215, 2: 213 },
                       "dutch": { 1:  22, 2:  27 },
                 "no answer 1": { 1: 402, 2: 319 },
                     "primary": { 1:   5, 2:   4 },
                       "lower": { 1:   0, 2:   0 },
                      "school": { 1:  56, 2:  33 },
                "intermediate": { 1: 103, 2:  68 },
                      "higher": { 1: 137, 2: 124 },
                  "university": { 1:  40, 2:  58 },
                 "no answer 2": { 1:  11, 2:  19 },
                         "yes": { 1:  25, 2:  27 },
                          "no": { 1: 308, 2: 240 },
                 "no answer 3": { 1:  91, 2:  79 },
                     "i think": { 1: 334, 2: 264 },
                      "i want": { 1:  14, 2:  10 },
                   "something": { 1:  36, 2:  31 },
                "others think": { 1:  14, 2:   6 },
               "other reasons": { 1:  24, 2:  29 },
                 "no answer 4": { 1:   2, 2:   6 },
                       "never": { 1: 164, 2: 187 },
                "now and then": { 1:  36, 2:  24 },
                       "daily": { 1: 224, 2: 135 },
             }

In [3]:
def pretty_print(results):
    results_list = []
    for key in results:
        if results[key][1] + results[key][2] == 0:
            print(f"pretty_print: skipping unused key value: {key}")
            continue
        percentage = round(100 * results[key][1] / (results[key][1] + results[key][2]), 1)
        results_list.append({"key": key, 
                             "dropout:N": results[key][1],
                             "dropout:%": percentage,
                             "completer:N": results[key][2],
                             "completer:%": 100 - percentage,
                            })
    return pd.DataFrame(results_list)

In [4]:
pretty_print(table_dict)

pretty_print: skipping unused key value: lower


,key,dropout:N,dropout:%,completer:N,completer:%
0,males,209,61.1,133,38.9
1,females,215,50.2,213,49.8
2,dutch,22,44.9,27,55.1
3,no answer 1,402,55.8,319,44.2
4,primary,5,55.6,4,44.4
5,school,56,62.9,33,37.1
6,intermediate,103,60.2,68,39.8
7,higher,137,52.5,124,47.5
8,university,40,40.8,58,59.2
9,no answer 2,11,36.7,19,63.3


## 2. Read data

There are 791 records in the file. 21 need to be removed because of missing information. The fuction `count_missing` identifies these.

In [5]:
DATAFILE = "/home/erikt/projects/e-mental-health/usb/releases/20200320/liwc+intake123+dropout.csv"

In [6]:
data = pd.read_csv(DATAFILE)

In [7]:
def count_missing(data):
    missings = []
    for idx, row in data.iterrows():
        missing = 0
        if pd.isna(row["1-geslacht"]) and pd.isna(row["1-geslacht0"]) and pd.isna(row["1-geslachtt0"]):
            missing += 1
        if pd.isna(row["3-national0"]) or row["3-national0"] == "EMPTY":
            missing += 1
        if pd.isna(row["4-opleidng"]) and (pd.isna(row["8-opleidng0"]) or row["8-opleidng0"] == "EMPTY") and pd.isna(row["4-opleidngt0"]):
            missing += 1
        if pd.isna(row["19-behdrink"]) and pd.isna(row["25-behdrinkt0"]):
            missing += 1
        if pd.isna(row["7-aanleid"]) and pd.isna(row["12-aanleid0"]) and pd.isna(row["7-aanleidt0"]):
            missing += 1
        if pd.isna(row["13-roken"]) and pd.isna(row["13-rokent0"]):
            missing += 1
        missings.append(missing)
    return missings

In [8]:
def count_values(missings):
    return pd.DataFrame([{"count": x} for x in missings])["count"].value_counts()

In [9]:
missings = count_missing(data)

In [10]:
count_values(missings)

1    721
2     49
4     21
Name: count, dtype: int64

## 3. Count gender

In [11]:
def count_gender(data, missings):
    genders = { "man": {1: 0, 2: 0}, "vrouw": {1: 0, 2: 0} }
    for idx, row in data.iterrows():
        if missings[idx] < 3:
            gender = math.nan
            if not pd.isna(row["1-geslacht"]):
                gender = row["1-geslacht"]
            if not pd.isna(row["1-geslacht0"]):
                if not pd.isna(gender):
                    print("cannot happen: duplicate gender value")
                gender = row["1-geslacht0"]
            if not pd.isna(row["1-geslachtt0"]):
                if not pd.isna(gender):
                    print("cannot happen: duplicate gender value")
                gender = row["1-geslachtt0"]
            if pd.isna(gender):
                print("cannot happen: missing gender value")
            genders[gender][row["dropout"]] += 1
    return genders

In [12]:
pretty_print(count_gender(data, missings))

,key,dropout:N,dropout:%,completer:N,completer:%
0,man,209,61.1,133,38.9
1,vrouw,215,50.2,213,49.8


## 4. Count nationalities

Value NaN stands for "Not a Number" and signals missing data

In [13]:
def count_nationalities(data, missings):
    nationalities = { "nederlands": {1: 0, 2: 0}, "EMPTY": {1: 0, 2: 0}, math.nan: {1: 0, 2: 0}}
    for idx, row in data.iterrows():
        if missings[idx] < 3:
            nationality = math.nan
            if not pd.isna(row["3-national0"]):
                nationality = row["3-national0"]
            nationalities[nationality][row["dropout"]] += 1
    return nationalities

In [14]:
pretty_print(count_nationalities(data, missings))

pretty_print: skipping unused key value: EMPTY


,key,dropout:N,dropout:%,completer:N,completer:%
0,nederlands,22,44.9,27,55.1
1,NaN,402,55.8,319,44.2


## 5. Count educations

Key value "MULTIWORD" represents a phrase removed by the anonymization process. It is unknown why some key fields have been filled with numbers (in this case: 3)

In [15]:
def count_educations(data, missings):
    educations = { "hbo": {1: 0, 2: 0}, "mbo": {1: 0, 2: 0}, "wo": {1: 0, 2: 0}, "lbo/mavo": {1: 0, 2: 0}, "havo/vwo": {1: 0, 2: 0}, 
                  "MULTIWORD": {1: 0, 2: 0}, "lbo/mavo/vmbo": {1: 0, 2: 0}, "basisschool": {1: 0, 2: 0}, "EMPTY": {1: 0, 2: 0}, "3": {1: 0, 2: 0}, }
    for idx, row in data.iterrows():
        if missings[idx] < 3:
            education = math.nan
            if not pd.isna(row["4-opleidng"]):
                education = row["4-opleidng"]
            if not pd.isna(row["8-opleidng0"]):
                if not pd.isna(education):
                    print(f"cannot happen: duplicate education value: {education} and {row['8-opleidng0']}")
                education = row["8-opleidng0"]
            if not pd.isna(row["4-opleidngt0"]):
                if not pd.isna(education):
                    print(f"cannot happen: duplicate education value: {education} and {row['4-opleidngt0']}")
                education = row["4-opleidngt0"]
            if pd.isna(education):
                print("cannot happen: missing education value")
            educations[education][row["dropout"]] += 1
    return educations

In [16]:
pretty_print(count_educations(data, missings))

pretty_print: skipping unused key value: EMPTY


,key,dropout:N,dropout:%,completer:N,completer:%
0,hbo,137,52.5,124,47.5
1,mbo,103,60.2,68,39.8
2,wo,40,40.8,58,59.2
3,lbo/mavo,65,65.0,35,35.0
4,havo/vwo,56,62.9,33,37.1
5,MULTIWORD,14,42.4,19,57.6
6,lbo/mavo/vmbo,4,50.0,4,50.0
7,basisschool,5,55.6,4,44.4
8,3,0,0.0,1,100.0


## 6. Count previous treatments

In [17]:
def count_treatments(data, missings):
    treatments = { "nee": {1: 0, 2: 0}, "ja": {1: 0, 2: 0}, "EMPTY": {1: 0, 2: 0}, math.nan: {1: 0, 2: 0},
                   "ja ik heb deze internetbehandeling al eens gevolgd.": {1: 0, 2: 0}, "MULTIWORD": {1: 0, 2: 0}, }
    for idx, row in data.iterrows():
        if missings[idx] < 3:
            treatment = math.nan
            if not pd.isna(row["19-behdrink"]):
                treatment = row["19-behdrink"]
            if not pd.isna(row["25-behdrinkt0"]):
                if not pd.isna(treatment):
                    print(f"cannot happen: duplicate treatment value: {reason} and {row['25-behdrinkt0']}")
                treatment = row["25-behdrinkt0"]
            if not pd.isna(row["26-behversl0"]):
                if not pd.isna(treatment):
                    print(f"cannot happen: duplicate treatment value: {reason} and {row['25-behversl0']}")
                treatment = row["26-behversl0"]

            treatments[treatment][row["dropout"]] += 1
    return treatments

In [18]:
pretty_print(count_treatments(data, missings))

pretty_print: skipping unused key value: nan


,key,dropout:N,dropout:%,completer:N,completer:%
0,nee,308,56.2,240,43.8
1,ja,12,50.0,12,50.0
2,EMPTY,91,53.5,79,46.5
3,ja ik heb deze internetbehandeling al eens gev...,13,50.0,13,50.0
4,MULTIWORD,0,0.0,2,100.0


## 7. Count reasons for treatment

In [19]:
def count_reasons(data, missings):
    reasons = { "ik vind zelf dat ik teveel drink": {1: 0, 2: 0}, 
                "anderen vinden dat ik teveel drink": {1: 0, 2: 0},
                "ik wil advies over mijn alcoholgebruik": {1: 0, 2: 0},
                "er is iets vervelends gebeurd en daarom wil ik iets aan mijn drinken doen": {1: 0, 2: 0},
                "1": {1: 0, 2: 0}, 
                "2": {1: 0, 2: 0}, 
                "3": {1: 0, 2: 0}, 
                "4": {1: 0, 2: 0}, 
                "MULTIWORD": {1: 0, 2: 0}, 
                "EMPTY": {1: 0, 2: 0}, }
    for idx, row in data.iterrows():
        if missings[idx] < 3:
            reason = math.nan
            if not pd.isna(row["7-aanleid"]):
                reason = row["7-aanleid"]
            if not pd.isna(row["12-aanleid0"]):
                if not pd.isna(reason):
                    print(f"cannot happen: duplicate education value: {reason} and {row['12-aanleid0']}")
                reason = row["12-aanleid0"]
            if not pd.isna(row["7-aanleidt0"]):
                if not pd.isna(reason):
                    print(f"cannot happen: duplicate education value: {reason} and {row['7-aanleidt0']}")
                reason = row["7-aanleidt0"]
            if pd.isna(reason):
                print("cannot happen: missing reason value")
            reasons[reason][row["dropout"]] += 1
    return reasons

In [20]:
pretty_print(count_reasons(data, missings))

,key,dropout:N,dropout:%,completer:N,completer:%
0,ik vind zelf dat ik teveel drink,334,55.9,264,44.1
1,anderen vinden dat ik teveel drink,14,70.0,6,30.0
2,ik wil advies over mijn alcoholgebruik,14,58.3,10,41.7
3,er is iets vervelends gebeurd en daarom wil ik...,36,53.7,31,46.3
4,1,1,100.0,0,0.0
5,2,2,40.0,3,60.0
6,3,1,100.0,0,0.0
7,4,1,50.0,1,50.0
8,MULTIWORD,19,43.2,25,56.8
9,EMPTY,2,25.0,6,75.0


## 8. Count smoking behaviour

In [21]:
def count_smokers(data, missings):
    smokers = { "ja af en toe": {1: 0, 2: 0}, "ja dagelijks": {1: 0, 2: 0}, "nee helemaal niet": {1: 0, 2: 0}, 
                math.nan: {1: 0, 2: 0}, "MULTIWORD": {1: 0, 2: 0}, "ja": {1: 0, 2: 0}, "nee": {1: 0, 2: 0}, }
    for idx, row in data.iterrows():
        if missings[idx] < 3:
            smoker = math.nan
            if not pd.isna(row["13-roken"]):
                smoker = row["13-roken"]
            if not pd.isna(row["19-tabak0"]):
                if not pd.isna(smoker):
                    print("cannot happen: duplicate smoker value")
                smoker = row["19-tabak0"]
            if not pd.isna(row["13-rokent0"]):
                if not pd.isna(smoker):
                    print("cannot happen: duplicate smoker value")
                smoker = row["13-rokent0"]
            smokers[smoker][row["dropout"]] += 1
    return smokers

In [22]:
pretty_print(count_smokers(data, missings))

pretty_print: skipping unused key value: nan


,key,dropout:N,dropout:%,completer:N,completer:%
0,ja af en toe,31,60.8,20,39.2
1,ja dagelijks,210,62.3,127,37.7
2,nee helemaal niet,159,48.3,170,51.7
3,MULTIWORD,2,50.0,2,50.0
4,ja,21,46.7,24,53.3
5,nee,1,25.0,3,75.0


## 9. Inspecting metadata field names

In [23]:
def find_columns(data, column_name):
    for column in data.columns:
        if re.search(column_name, column, flags=re.IGNORECASE):
            print(column)

In [24]:
find_columns(data, "tab")

19-tabak0
19-tabakjr0
20-tabak300
20-tabakdg0
20-tabakhv0
21-tabak0h


In [25]:
data["19-tabak0"].value_counts()

ja       45
EMPTY    21
nee       4
Name: 19-tabak0, dtype: int64

In [26]:
for column in data.columns:
    print(column)

file
nbr of mails
Number of matches
number count
1 function
2 pronoun
3 ppron
4 i
5 we
6 you
7 shehe
8 they
9 ipron
10 article
11 prep
12 auxverb
13 adverb
14 conj
15 negate
20 verb
21 adj
22 compare
23 interrog
24 number
25 quant
30 affect
31 posemo
32 negemo
33 anx
34 anger
35 sad
40 social
41 family
42 friend
43 female
44 male
50 cogproc
51 insight
52 cause
53 discrep
54 tentat
55 certain
56 differ
60 percept
61 see
62 hear
63 feel
70 bio
71 body
72 health
73 sexual
74 ingest
80 drives
81 affiliation
82 achieve
83 power
84 reward
85 risk
90 focuspast
91 focuspresent
92 focusfuture
100 relativ
101 motion
102 space
103 time
110 work
111 leisure
112 home
113 money
114 relig
115 death
120 informal
121 swear
122 netspeak
123 assent
124 nonflu
125 filler
counselor
dropout
0-title
1-geslacht
1-geslacht0
1-geslachtt0
10-dag1
10-dag1t0
10-dag2
10-dag2t0
10-dag3
10-dag3t0
10-dag4
10-dag4t0
10-dag5
10-dag5t0
10-dag6
10-dag6t0
10-dag7
10-dag7t0
10-dagb0
10-typisch
10-typischt0
10-week
10-weekt0